In [4]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

## End goal is to understand how to implement Machine Learning Pipelines

In [5]:
df = pd.read_csv("https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/main/day29-sklearn-pipelines/train.csv")

In [6]:
df.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
np.round(df.describe(), 2)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.00,891.00,891.00,714.00,891.00,891.00,891.00
mean,446.00,0.38,2.31,29.70,0.52,0.38,32.20
std,257.35,0.49,0.84,14.53,1.10,0.81,49.69
min,1.00,0.00,1.00,0.42,0.00,0.00,0.00
25%,223.50,0.00,2.00,20.12,0.00,0.00,7.91
50%,446.00,0.00,3.00,28.00,0.00,0.00,14.45
75%,668.50,1.00,3.00,38.00,1.00,0.00,31.00
max,891.00,1.00,3.00,80.00,8.00,6.00,512.33


### Find null present or not

In [8]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [9]:
### Age Cabin and Embarked have null values in it

### We have to determine whether a passenger survived or not

In [11]:
df.drop(columns = ["PassengerId", "Name", "Ticket", "Cabin"], inplace = True)

In [12]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


### Train Test Split

In [13]:
x_train, x_test, y_train, y_test = train_test_split(df.drop(columns = ["Survived"], axis = 1), df["Survived"], 
                                                    test_size = 0.2, random_state = 0)

In [15]:
x_train.shape, x_test.shape

((712, 7), (179, 7))

In [16]:
x_train.isnull().sum()

Pclass        0
Sex           0
Age         141
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

### Applying Imputation

In [41]:
si_age = SimpleImputer()

si_embarked = SimpleImputer(strategy = 'most_frequent')

x_train_age = si_age.fit_transform(x_train[["Age"]])

x_test_age = si_age.fit_transform(x_test[["Age"]])

x_train_embarked = si_embarked.fit_transform(x_train[["Embarked"]])
x_test_embarked = si_embarked.fit_transform(x_test[["Embarked"]])
x_train_embarked.shape

(712, 1)

### Applying OneHotEncoding

In [42]:
ohe_sex = OneHotEncoder(sparse = False, handle_unknown = 'ignore')
ohe_embarked = OneHotEncoder(sparse = False, handle_unknown = 'ignore')

# Since we are applying DecisionTree so multicollinearity doesnt effect the algorithm

x_train_sex = ohe_sex.fit_transform(x_train[["Sex"]])
x_test_sex = ohe_sex.fit_transform(x_test[["Sex"]])

x_train_embarked = ohe_embarked.fit_transform(x_train_embarked)
x_test_embarked = ohe_embarked.fit_transform(x_test_embarked)

In [43]:
x_train_sex, x_train_embarked.shape

(array([[1., 0.],
        [0., 1.],
        [0., 1.],
        ...,
        [0., 1.],
        [1., 0.],
        [0., 1.]]),
 (712, 3))

In [28]:
x_train_rem = x_train.drop(columns = ["Sex", "Age", "Embarked"], axis = 1)

x_test_rem = x_test.drop(columns = ["Sex", "Age", "Embarked"], axis = 1)

In [29]:
x_train_rem.head()

,Pclass,SibSp,Parch,Fare
140,3,0,2,15.2458
439,2,0,0,10.5000
817,2,1,1,37.0042
378,3,0,0,4.0125
491,3,0,0,7.2500


### Concatenate the final dataframe

In [32]:
x_train_transformed = np.concatenate((x_train_rem, x_train_age, x_train_sex, x_train_embarked), axis = 1)

x_test_transformed = np.concatenate((x_test_rem, x_test_age, x_test_sex, x_test_embarked), axis = 1)

In [38]:
x_train_transformed.shape

(712, 10)

## Now all the transformations which we have done above we will do with the help of a cloumn transformer

In [44]:
si_embarked = SimpleImputer(strategy = 'most_frequent')

x_train_embarked = si_embarked.fit_transform(x_train[["Embarked"]])
x_test_embarked = si_embarked.fit_transform(x_test[["Embarked"]])

In [48]:
x_train_embarked = pd.DataFrame(x_train_embarked, columns = ["Embarked"])
x_test_embarked = pd.DataFrame(x_test_embarked, columns = ["Embarked"])
x_train_embarked.head()

,Embarked
0,C
1,S
2,C
3,C
4,S


In [49]:
from sklearn.compose import ColumnTransformer

transformer = ColumnTransformer(transformers = 
[
    ('tnf1', OneHotEncoder(sparse = True, handle_unknown = 'ignore'), ["Sex"]),
    ('tnf2', SimpleImputer(), ["Age"])
],
  remainder = 'passthrough')

x_train_sex_age = transformer.fit_transform(x_train.drop("Embarked", axis = 1))
x_test_sex_age = transformer.fit_transform(x_test.drop("Embarked", axis = 1))

In [50]:
x_train_sex_age.shape

(712, 7)

In [54]:
transformer_new = ColumnTransformer(transformers = 
                               [('tnf3',OneHotEncoder(sparse = False, handle_unknown = 'ignore'),["Embarked"])
                                ], remainder = 'passthrough')

x_train_embarked_modified = transformer_new.fit_transform(x_train_embarked)
x_test_embarked_modified = transformer_new.fit_transform(x_test_embarked)

In [56]:
x_train_embarked_modified.shape

(712, 3)

### Concatenate all th np arrays

In [57]:
x_train_modified = np.concatenate((x_train_sex_age,x_train_embarked_modified), axis = 1)

x_test_modified = np.concatenate((x_test_sex_age,x_test_embarked_modified), axis = 1)

In [58]:
x_train_modified.shape

(712, 10)

In [59]:
x_train_modified

array([[ 1.        ,  0.        , 29.74518389, ...,  1.        ,
         0.        ,  0.        ],
       [ 0.        ,  1.        , 31.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.        ,  1.        , 31.        , ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  1.        , 29.74518389, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.        ,  0.        , 36.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 0.        ,  1.        , 60.        , ...,  0.        ,
         0.        ,  1.        ]])

## DecisionTree

In [60]:
clf = DecisionTreeClassifier()
clf.fit(x_train_modified, y_train)

DecisionTreeClassifier()

## Prediction

In [61]:
y_pred = clf.predict(x_test_modified)

In [62]:
y_pred

array([0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       1, 0, 1], dtype=int64)

## Accuracy of model

In [63]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test,y_pred)

0.770949720670391

## In order to deploy model, we need pickle library

In [64]:
import pickle

In [65]:
### We will deal with this at a later stage, for now we have to work on model building side only